# CQL

In [ ]:
        # cql args
        # cql_kwargs = args['cql_kwargs']
        # self.use_cql = cql_kwargs['use_cql']
        # self.num_random = cql_kwargs['num_random']
        # self.cql_weight = cql_kwargs['cql_weight']
        # self.with_lagrange = cql_kwargs['with_lagrange']
        # lagrange_thresh = cql_kwargs['lagrange_thresh']
        # if lagrange_thresh < 0:
        #     self.with_lagrange = False
        # if self.with_lagrange:
        #     self.target_action_gap = lagrange_thresh
        #     self.log_alpha_prime = torch.zeros(1, device=self.device, requires_grad=True)
        #     self.alpha_prime_optimizer = Adam([self.log_alpha_prime], lr=zf_lr)

In [ ]:
"""
CQL (new mate)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    
    _, tau_hat, _ = self.get_tau(policy_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]
    
    z1_presum_tau, z2_presum_tau = self._get_tensor_values(policy_obs, presum_tauom_actions, presum_tauom_tau_hat)

    z1_presum_tau = torch.logsumexp(z1_presum_tau, dim=1).mean()
    z2_presum_tau = torch.logsumexp(z2_presum_tau, dim=1).mean()
    cql_z1_loss = (z1_presum_tau - expert_z1_pred.mean()) * self.cql_weight
    cql_z2_loss = (z2_presum_tau - expert_z2_pred.mean()) * self.cql_weight
    
    zf1_loss += cql_z1_loss
    zf2_loss += cql_z2_loss

In [ ]:
"""
CQL (old mate)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    current_actions, current_log_pi = self._get_policy_actions(policy_obs, 
                                                               num_actions=self.num_presum_tauom,
                                                               network=self.policy)
    next_actions, next_log_pi = self._get_policy_actions(policy_next_obs, 
                                                         num_actions=self.num_presum_tauom,
                                                         network=self.policy)
    
    _, tau_hat, _ = self.get_tau(policy_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]

    z1_presum_tau, z2_presum_tau = self._get_tensor_values(policy_obs, presum_tauom_actions, presum_tauom_tau_hat)
    z1_current, z2_current = self._get_tensor_values(policy_obs, current_actions, presum_tauom_tau_hat)
    z1_next, z2_next = self._get_tensor_values(policy_obs, next_actions, presum_tauom_tau_hat)

    presum_tauom_density = np.log(0.5 ** current_actions.shape[-1])
    cat_z1 = torch.cat(
        [z1_presum_tau - alpha * presum_tauom_density, 
         z1_next - alpha * next_log_pi.detach(),
         z1_current - alpha * current_log_pi.detach()], 1
    )
    cat_z2 = torch.cat(
        [z2_presum_tau - alpha * presum_tauom_density, 
         z2_next - alpha * next_log_pi.detach(),
         z2_current - alpha * current_log_pi.detach()], 1
    )
    min_zf1_loss = torch.logsumexp(cat_z1, dim=1).mean()
    min_zf2_loss = torch.logsumexp(cat_z2, dim=1).mean()
    min_zf1_loss = (min_zf1_loss - expert_z1_pred.mean()) * self.min_z_weight
    min_zf2_loss = (min_zf2_loss - expert_z2_pred.mean()) * self.min_z_weight
    
    if self.with_lagrange:
        alpha_prime = torch.clamp(self.log_alpha_prime.exp(), min=0.0, max=1000000.0)
        min_zf1_loss = alpha_prime * (min_zf1_loss - self.target_action_gap)
        min_zf2_loss = alpha_prime * (min_zf2_loss - self.target_action_gap)

        self.alpha_prime_optimizer.zero_grad()
        alpha_prime_loss = (-min_zf1_loss - min_zf2_loss) * 0.5
        alpha_prime_loss.backward(retain_graph=True)
        self.alpha_prime_optimizer.step()

    zf1_loss += min_zf1_loss
    zf2_loss += min_zf2_loss

In [ ]:
        """
        CQL (std!)
        """
        if self.use_cql:
            # q1_pred = torch.sum(presum_tau * policy_z1_pred, dim=1, keepdims=True)
            # q2_pred = torch.sum(presum_tau * policy_z2_pred, dim=1, keepdims=True)
            # q1_std = presum_tau * (policy_z1_pred - q1_pred).pow(2)
            # q2_std = presum_tau * (policy_z2_pred - q2_pred).pow(2)
            # cql1_term = self.cql_weight * q1_std.sum(dim=1).sqrt().mean()
            # cql2_term = self.cql_weight * q2_std.sum(dim=1).sqrt().mean()

            cql1_term = 4.5 - self.cql_weight * policy_z1_pred.std()
            cql2_term = 4.5 - self.cql_weight * policy_z2_pred.std()
            
            zf1_loss += cql1_term
            zf2_loss += cql2_term

In [ ]:
"""
CQL (old mate; expert observations with policy actions)
"""
if self.use_cql:
    presum_tauom_actions = torch.FloatTensor(policy_actions.shape[0] * self.num_presum_tauom,
                                       policy_actions.shape[-1]).uniform_(-1,1).to(self.device)
    current_actions, current_log_pi = self._get_policy_actions(expert_obs, 
                                                               num_actions=self.num_presum_tauom,
                                                               network=self.policy)
    next_actions, next_log_pi = self._get_policy_actions(expert_next_obs, 
                                                         num_actions=self.num_presum_tauom,
                                                         network=self.policy)
    
    _, tau_hat, _ = self.get_tau(expert_obs, policy_actions, fp=self.fp)
    tau_index = np.presum_tauom.presum_tauint(0, self.num_quantiles)
    presum_tauom_tau_hat = tau_hat[:, tau_index:tau_index+1]

    z1_presum_tau, z2_presum_tau = self._get_tensor_values(expert_obs, presum_tauom_actions, presum_tauom_tau_hat)
    z1_current, z2_current = self._get_tensor_values(expert_obs, current_actions, presum_tauom_tau_hat)
    z1_next, z2_next = self._get_tensor_values(expert_obs, next_actions, presum_tauom_tau_hat)

    presum_tauom_density = np.log(0.5 ** current_actions.shape[-1])
    cat_z1 = torch.cat(
        [z1_presum_tau - alpha * presum_tauom_density, 
         z1_next - alpha * next_log_pi.detach(),
         z1_current - alpha * current_log_pi.detach()], 1
    )
    cat_z2 = torch.cat(
        [z2_presum_tau - alpha * presum_tauom_density, 
         z2_next - alpha * next_log_pi.detach(),
         z2_current - alpha * current_log_pi.detach()], 1
    )
    min_zf1_loss = torch.logsumexp(cat_z1, dim=1).mean() * self.cql_weight
    min_zf2_loss = torch.logsumexp(cat_z2, dim=1).mean() * self.cql_weight

    zf1_loss += min_zf1_loss
    zf2_loss += min_zf2_loss

# alpha decay

In [ ]:
        # linear schedule entropy temperature
        # self.start_alpha = 0.01
        # self.end_alpha = 0.0001
        # self.duration = 150000
        # self.slope = (self.end_alpha - self.start_alpha) / self.duration
        # self.alpha_decay(0)

In [ ]:
    # def alpha_decay(self, epoch):
    #     self.alpha = max(self.slope * epoch + self.start_alpha, self.end_alpha)

    # @property
    # def alpha(self):
    #     return self.log_alpha.exp()

# OOD

In [ ]:
        # Monitor: OOD action values
        # policy_z1_new, policy_z2_new = self.getZ(policy_obs, new_actions, tau_hat)
        # policy_q1_new = torch.sum(presum_tau * policy_z1_new, dim=1, keepdims=True)
        # policy_q2_new = torch.sum(presum_tau * policy_z2_new, dim=1, keepdims=True)
        # q1_ood = (policy_q1_new - policy_q1_pred) / (torch.abs(policy_q1_pred) + self.EPS)
        # q2_ood = (policy_q2_new - policy_q2_pred) / (torch.abs(policy_q2_pred) + self.EPS)

# TQC

In [ ]:
def experiment_tqc(variant):
    dummy_env = make_env(variant['env'])
    obs_dim = dummy_env.observation_space.low.size
    action_dim = dummy_env.action_space.low.size
    expl_env = VectorEnv([lambda: make_env(variant['env']) for _ in range(variant['expl_env_num'])])
    expl_env.seed(variant["seed"])
    expl_env.action_space.seed(variant["seed"])
    eval_env = SubprocVectorEnv([lambda: make_env(variant['env']) for _ in range(variant['eval_env_num'])])
    eval_env.seed(variant["seed"])

    M = variant['layer_size']
    num_quantiles = variant['num_quantiles']
    n_nets = variant['n_nets']
    
    zf = Critic(
        input_size=obs_dim + action_dim,
        output_size=1,
        num_quantiles=num_quantiles,
        hidden_sizes=[M, M],
        n_nets=n_nets,
    )
    target_zf = Critic(
        input_size=obs_dim + action_dim,
        output_size=1,
        num_quantiles=num_quantiles,
        hidden_sizes=[M, M],
        n_nets=n_nets,
    )
    policy = TanhGaussianPolicy(
        obs_dim=obs_dim,
        action_dim=action_dim,
        hidden_sizes=[M, M],
    )
    eval_policy = MakeDeterministic(policy)
    # fraction proposal network
    fp = target_fp = None
    if variant['trainer_kwargs'].get('tau_type') == 'fqf':
        fp = FlattenMlp(
            input_size=obs_dim + action_dim,
            output_size=num_quantiles,
            hidden_sizes=[M // 2, M // 2],
            output_activation=softmax,
        )
        target_fp = FlattenMlp(
            input_size=obs_dim + action_dim,
            output_size=num_quantiles,
            hidden_sizes=[M // 2, M // 2],
            output_activation=softmax,
        )
    eval_path_collector = VecMdpPathCollector(
        eval_env,
        eval_policy,
    )
    expl_path_collector = VecMdpStepCollector(
        expl_env,
        policy,
    )
    replay_buffer = TorchReplayBuffer(
        variant['replay_buffer_size'],
        dummy_env,
    )
    expert_buffer = TorchReplayBuffer(
        variant['replay_buffer_size'] // 10,
        dummy_env,
    )
    iq_args = variant['iq_kwargs']
    expert_buffer.load(iq_args['expert_path'], iq_args['demos'], 
                       iq_args['subsample_freq'], variant['seed']
                      )
    trainer = TruncIDSACTrainer(
        args=variant,
        env=dummy_env,
        policy=policy,
        zf=zf,
        target_zf=target_zf,
        fp=fp,
        target_fp=target_fp,
        num_quantiles=num_quantiles,
        **variant['trainer_kwargs'],
    )
    algorithm = TorchVecOnlineIQAlgorithm(
        trainer=trainer,
        exploration_env=expl_env,
        evaluation_env=eval_env,
        exploration_data_collector=expl_path_collector,
        evaluation_data_collector=eval_path_collector,
        replay_buffer=replay_buffer,
        expert_buffer=expert_buffer,
        **variant['algorithm_kwargs'],
    )
    algorithm.to(ptu.device)
    algorithm.train()


# iq_distr_loss

In [ ]:
def iq_distr_loss(
    input_expert, 
    target_expert,
    input_policy,
    target_policy,
    policy_reward,
    log_pi,
    alpha,
    tau,
    weight,
    args
):
    expert_reward = input_expert - target_expert
    policy_reward = input_policy - alpha * log_pi - target_policy
    loss = - expert_reward.mean() + policy_reward.mean()

    if args['regularize']:
        expert_rho = quantile_regression_loss(input_expert, 10 + target_expert, tau, weight)
        policy_rho = quantile_regression_loss(input_policy, policy_reward + target_policy, tau, weight)
        chi2_loss = 1/(4 * args['alpha']) * torch.cat([expert_rho, policy_rho], dim=-1).mean()
        loss += chi2_loss
    
    loss_dict = {
        'expert_reward': expert_reward.mean().item(),
        'policy_reward': policy_reward.mean().item(),
        'chi2_loss': chi2_loss.item() if args['regularize'] else 0,
    }
    return loss, loss_dict


def quantile_regression_loss(inputs, targets, tau, weight):
    """
    input: (N, T)
    target: (N, T)
    tau: (N, T)
    """
    inputs = inputs.unsqueeze(-1)
    targets = targets.detach().unsqueeze(-2)
    tau = tau.detach().unsqueeze(-1)
    # weight = weight.detach().unsqueeze(-2)
    expanded_inputs, expanded_targets = torch.broadcast_tensors(inputs, targets)
    L = (expanded_inputs - expanded_targets)**2
    sign = torch.sign(expanded_inputs - expanded_targets) / 2. + 0.5
    rho = torch.abs(tau - sign) * L
    return rho.sum(dim=-1)

# imports

In [2]:
import numpy as np
import torch
import gym
import csv
from rlkit.envs import make_env
from rlkit.envs.vecenv import SubprocVectorEnv, VectorEnv
from rlkit.torch.idsac.networks import QuantileMlp, Critic
from rlkit.torch.idsac.policies import TanhGaussianPolicy
from rlkit.data_management.torch_replay_buffer import TorchReplayBuffer
import rlkit.torch.pytorch_util as ptu
from rlkit.torch.core import eval_np

In [6]:
x = torch.tensor([10], dtype=torch.float, requires_grad=True).to(torch.device('cuda'))

In [7]:
x

tensor([10.], device='cuda:0', grad_fn=<ToCopyBackward0>)

In [8]:
x + 1001

tensor([1011.], device='cuda:0', grad_fn=<AddBackward0>)